In [ ]:
from __future__ import annotations

from typing import Literal

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.axes import Axes
from matplotlib.figure import Figure

from src.drawing_utils import Point, Segment
from src.plotting_utils import configure_matplotlib, rm
from src.supply_demand import (
    DX,
    Curve,
    DemandCurve,
    SupplyCurve,
    SupplyDemand,
)
from src.supply_demand_plotter import CostUtilityPlotter, SupplyDemandPlotter

configure_matplotlib()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(6.4, 4.2), layout="tight")

x_vals = np.linspace(0, 10)
supply_demand = SupplyDemand(
    [SupplyCurve([Point(x, 1 + 1 * x) for x in x_vals], stepped=False)],
    [DemandCurve([Point(x, 10 - x) for x in x_vals], stepped=False)],
    equilibrium_price=5.5,
)
SupplyDemandPlotter(ax1).plot_all(supply_demand)
Segment(supply_demand.equilibrium, Point(5.5, 5.5)).drawn(ax1).end.labeled(
    ax1, "$(Q^*, P^*)$", ha="left", va="center"
)
CostUtilityPlotter(ax2, ylim=(0, 60)).plot_all(supply_demand)

fig.savefig("img/fig_2_1.png", dpi=200)

### Example 2.1

In [ ]:
supply_demand = SupplyDemand(
    [SupplyCurve([Point(0, 0), Point(6, 2), Point(9, 7)])],
    [DemandCurve([Point(0, 8), Point(4, 8), Point(8, 5)])],
    equilibrium_price=5,
)
equilibrium_quantity = supply_demand.equilibrium_quantity()
print(f"Q_opt = {equilibrium_quantity:.3f}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(6.4, 4.2), layout="tight")

SupplyDemandPlotter(ax1).plot_all(supply_demand)
Segment(supply_demand.equilibrium, (6.5, 4)).drawn(ax1).end.labeled(
    ax1, "$(Q^*, P^*)$", ha="left", va="top"
)

CostUtilityPlotter(ax2, ylim=(0, 50)).plot_all(
    supply_demand, equilibrium_quantity=equilibrium_quantity
)

fig.savefig("img/fig_2_2.png", dpi=200)

### Example 2.2

In [ ]:
cost = r"{name} Cost", r"$C_\mathrm{{name}}$"
utility = r"{name} Utility", r"$U_\mathrm{{name}}$"
dashed = "--"
dotted = ":"

supply_demand = SupplyDemand(
    supply_curves=[
        SupplyCurve([Point(0, 0), Point(6, 2), Point(9, 7)], "G1", *cost, fmt=dashed),
        SupplyCurve([Point(0, 0), Point(7, 4), Point(10, 10)], "G2", *cost, fmt=dotted),
    ],
    demand_curves=[
        DemandCurve(
            [Point(0, 8), Point(4, 8), Point(8, 5)], "L1", *utility, fmt=dashed
        ),
        DemandCurve(
            [Point(0, 9), Point(3, 9), Point(9, 3)], "L2", *utility, fmt=dotted
        ),
    ],
    equilibrium_price=4,
)

equilibrium_quantity = supply_demand.equilibrium_quantity()
print(f"Q_opt = {equilibrium_quantity:.3f}")

In [ ]:
def two_generators_two_loads(style: Literal["ticks", "area"]) -> Figure:
    fig, axs = plt.subplots(nrows=2, ncols=2, layout="tight")

    for ax, curve in zip(np.ndarray.flatten(axs), supply_demand.curves, strict=True):
        equilibrium_quantity = (
            supply_demand.equilibrium_quantity(curve.name) if style == "area" else None
        )
        if equilibrium_quantity is not None:
            print(f"Q_{curve.name}_opt = {equilibrium_quantity}")
        plotter = SupplyDemandPlotter(
            ax,
            xticks=(
                {equilibrium_quantity: rf"$Q_\mathrm{{{curve.name}}}^*$"}
                if style == "area"
                else None
            ),
            yticks=({} if style == "area" else None),
        )
        plotter.plot(curve, equilibrium_quantity)
        ax.set_title(rm(curve.name))

    return fig, axs

In [ ]:
fig, _ = two_generators_two_loads(style="ticks")
fig.savefig("img/fig_2_3.png", dpi=200)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = two_generators_two_loads(style="area")

Point(3, 1).labeled(ax1, r"$C_\mathrm{G1}(Q_\mathrm{G1}^*)$")
Point(2.5, 2).labeled(ax2, r"$C_\mathrm{G2}(Q_\mathrm{G2}^*)$")
Point(4, 3.25).labeled(ax3, r"$U_\mathrm{L2}(Q_\mathrm{L2}^*)$")
Point(1.5, 4.5).labeled(ax4, r"$U_\mathrm{L2}(Q_\mathrm{L2}^*)$")

fig.savefig("img/fig_2_4.png", dpi=200)

In [ ]:
def add_horizontal_brace(
    ax: Axes, x1: float, x2: float, y: float, label: str, opening: Literal["up", "down"]
) -> None:
    center_x = (x2 + x1) / 2
    sign = {"up": -1, "down": 1}[opening]
    ax.annotate(
        label,
        xy=(center_x, y + 0.5 * sign),
        xytext=(center_x, y + 1.5 * sign),
        ha="center",
        va={"up": "top", "down": "bottom"}[opening],
        arrowprops=dict(arrowstyle=f"-[, widthB={(x2 - x1) / 2 * 1.06 - 0.2}"),
    )

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(
    nrows=4,
    figsize=(6.4, 6.4),
    layout="tight",
    gridspec_kw=dict(height_ratios=[1 / 3, 1 / 6, 1 / 6, 1 / 6]),
)
xlim = (0, 20)

SupplyDemandPlotter(ax1, xlim).plot_all(supply_demand)
add_horizontal_brace(ax1, x1=0, x2=6, y=2, label=r"$Q_\mathrm{G1}^*$", opening="down")
add_horizontal_brace(ax1, x1=6, x2=11, y=4, label=r"$Q_\mathrm{G2}^*$", opening="up")
add_horizontal_brace(ax1, x1=0, x2=3, y=9, label=r"$Q_\mathrm{L1}^*$", opening="down")
add_horizontal_brace(ax1, x1=3, x2=11, y=8, label=r"$Q_\mathrm{L2}^*$", opening="down")
Segment(supply_demand.equilibrium, Point(14, 5)).drawn(ax1).end.labeled(
    ax1, "$(Q^*, P^*)$", ha="left"
)

ylim = (0, 100)
CostUtilityPlotter(ax2, xlim, ylim).plot_multiple(
    supply_demand.supply_curves, total=True, equilibrium_quantity=equilibrium_quantity
)
CostUtilityPlotter(ax3, xlim, ylim).plot_multiple(
    supply_demand.demand_curves, total=True, equilibrium_quantity=equilibrium_quantity
)
CostUtilityPlotter(ax4, xlim, ylim).plot_welfare(supply_demand)

fig.savefig("img/fig_2_5.png", dpi=200)

### Example 2.3

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, figsize=(6.4, 6.4), layout="tight")
xlim = (0, 20)
quantity_vals = np.arange(xlim[0], xlim[1] + DX, DX)

transmission_line_efficiency = 0.75


def plot_cost_and_utility(ax: Axes, key: Literal["G", "L"]) -> None:
    cost_vals = {
        "G": supply_demand.cost()(quantity_vals),
        "L": supply_demand.cost()(quantity_vals / transmission_line_efficiency),
    }[key]
    utility_vals = {
        "G": supply_demand.utility()(quantity_vals * transmission_line_efficiency),
        "L": supply_demand.utility()(quantity_vals),
    }[key]
    welfare_vals = utility_vals - cost_vals
    idxmax = np.nanargmax(welfare_vals)
    equilibrium_quantity = quantity_vals[idxmax]
    print(f"Q_{key}_opt = {equilibrium_quantity}")

    label = rf"Q_\mathrm{{{key}}}"
    plotter = CostUtilityPlotter(
        ax,
        xlim,
        ylim=(0, 100),
        xticks={equilibrium_quantity: rf"${label}^*$"},
        yticks={},
        xaxis_label=rf"${label}$",
    )
    cost_or_utility = {
        "G": supply_demand.cost,
        "L": supply_demand.utility,
    }[key]
    for i in [1, 2]:
        [curve] = [c for c in supply_demand.curves if c.name == f"{key}{i}"]
        plotter.plot_cost_or_utility_vals(
            quantity_vals,
            cost_or_utility(curve.name)(quantity_vals),
            curve,
            equilibrium_quantity,
        )
    plotter.plot_cost_or_utility_vals(
        quantity_vals,
        cost_vals,
        SupplyCurve,
        equilibrium_quantity,
        label="Total Cost, $C$",
    )
    plotter.plot_cost_or_utility_vals(
        quantity_vals,
        utility_vals,
        DemandCurve,
        equilibrium_quantity,
        label="Total Utility, $U$",
    )
    plotter.plot_welfare_vals(quantity_vals, welfare_vals, equilibrium_quantity)
    plotter.legend()

    return equilibrium_quantity


generator_equilibrium_quantity = plot_cost_and_utility(ax2, "G")
load_equilibrium_quantity = plot_cost_and_utility(ax3, "L")

plotter = SupplyDemandPlotter(ax1, xlim, xticks={}, yticks={})
plotter.plot(
    Curve.composite(supply_demand.supply_curves),
    equilibrium_quantity=generator_equilibrium_quantity,
)
plotter.plot(
    Curve.composite(supply_demand.demand_curves),
    equilibrium_quantity=load_equilibrium_quantity,
)
plotter.legend()
Segment(Point(generator_equilibrium_quantity, 4).drawn(ax1), Point(10.5, 7)).drawn(
    ax1
).end.labeled(ax1, r"$(Q_\mathrm{G}^*, P_\mathrm{G}^*)$", va="bottom")
Segment(Point(load_equilibrium_quantity, 8).drawn(ax1), Point(8, 10)).drawn(
    ax1
).end.labeled(ax1, r"$(Q_\mathrm{L}^*, P_\mathrm{L}^*)$", va="bottom")

fig.savefig("img/fig_2_6.png", dpi=200)

### Example 2.4

In [ ]:
g1_cost = supply_demand.curve("G1").integral
g2_cost = supply_demand.curve("G2").integral
l_utility = supply_demand.utility()


def g1_quantity(g2_quantity: float, l_quantity: float) -> float:
    return (l_quantity - g2_quantity) / transmission_line_efficiency


def welfare(g2_quantity: float, l_quantity: float) -> float:
    return (
        l_utility(l_quantity)
        - g1_cost(g1_quantity(g2_quantity, l_quantity))
        - g2_cost(g2_quantity)
    )


dx = 0.01
g2_quantity_vals = l_quantity_vals = np.arange(0, 20 + dx, dx)
X, Y = np.meshgrid(g2_quantity_vals, l_quantity_vals)
Z = welfare(X, Y)
maximum_welfare = np.nanmax(Z)
idxmax = np.nanargmax(Z)
optimal_g2_quantity = np.ndarray.flatten(X)[idxmax]
optimal_l_quantity = np.ndarray.flatten(Y)[idxmax]
print(f"Q_G2_opt = {optimal_g2_quantity}")
print(f"Q_L_opt = {optimal_l_quantity}")
print(f"Q_G1_opt = {g1_quantity(optimal_g2_quantity, optimal_l_quantity)}")


fig, (ax1, ax2) = plt.subplots(
    ncols=2, figsize=(6.4, 3.2), subplot_kw=dict(projection="3d")
)


def plot(ax: Axes) -> None:
    ax.set_xlim(0, 20)
    ax.set_ylim(0, 20)
    ax.set_zlim(0, 50)
    ax.plot_surface(X, Y, Z, cmap="turbo")
    ax.plot(
        optimal_g2_quantity,
        [0, 0, 20],
        [0, maximum_welfare, maximum_welfare],
        "k:",
        zorder=2.5,
    )
    ax.plot(
        [0, 0, 20],
        optimal_l_quantity,
        [0, maximum_welfare, maximum_welfare],
        "k:",
        zorder=2.5,
    )
    ax.set_xlabel(r"$Q_\mathrm{G2}$")
    ax.set_ylabel(r"$Q_\mathrm{L}$")
    ax.set_zlabel(r"$W$")
    ax2.scatter(
        optimal_g2_quantity,
        optimal_l_quantity,
        maximum_welfare,
        "o",
        color="white",
    )


plot(ax1)
ax1.view_init(elev=45, azim=225)
ax1.set_title(rm("(a)"))
plot(ax2)
ax2.view_init(elev=90, azim=270)
ax2.set_proj_type("ortho")
ax2.set_zlabel(None)
ax2.set_zticks([])
ax2.set_title(rm("(b)"))
ax2.plot(
    [optimal_g2_quantity, 9], [optimal_l_quantity, 7], "k", linewidth=0.5, zorder=4.5
)
ax2.text(9, 7, maximum_welfare, rm("Optimum"), ha="left", va="top")

fig.savefig("img/fig_2_7.png", dpi=200)